In [2]:
import numpy as np
from itertools import combinations
import random
from scipy.stats import rankdata
from timeit import default_timer as timer
import matplotlib.pyplot as plt

random.seed(2021)

# World ranking points obtained mostly from https://web.archive.org/web/20210704140714/https://fiba3x3.com/en/rankings/individual.html

countries = ["Belgium", "China", "Japan", "Latvia","Netherlands", "Poland", "ROC", "Serbia"]
ranking_points = {}
ranking_points["Belgium"] = [150981,269580,19330,224490] 
ranking_points["China"] = [168242,5400,170280,12600] 
ranking_points["Japan"] = [78827,7006,6961,241224]
ranking_points["Latvia"] = [825366,735840,733800,664146]
ranking_points["Netherlands"] = [481692,15487,255145,15810]
ranking_points["Poland"] = [129899,118883,103807,302319]
ranking_points["ROC"] = [211700,279796,253640,225302]
ranking_points["Serbia"] = [788975,592650,788507,827172]

def ranking_2_probability(country_1,country_2):
    sum_1 = float(sum(ranking_points[country_1]))
    sum_2 = float(sum(ranking_points[country_2]))
    probability = sum_1 / (sum_1 + sum_2)
    return probability

In [3]:
# The games are combinations of the participants
games = list(combinations(countries,2))

# We precompute the probabilities for each game
probabilities = np.zeros(len(games))
for i, game in enumerate(games):
    probabilities[i] = ranking_2_probability(game[0],game[1])
    print("The probability that " + str(game[0]) +" beats " + str(game[1]) +" equals {:0.2f}".format(probabilities[i]))

The probability that Belgium beats China equals 0.65
The probability that Belgium beats Japan equals 0.67
The probability that Belgium beats Latvia equals 0.18
The probability that Belgium beats Netherlands equals 0.46
The probability that Belgium beats Poland equals 0.50
The probability that Belgium beats ROC equals 0.41
The probability that Belgium beats Serbia equals 0.18
The probability that China beats Japan equals 0.52
The probability that China beats Latvia equals 0.11
The probability that China beats Netherlands equals 0.32
The probability that China beats Poland equals 0.35
The probability that China beats ROC equals 0.27
The probability that China beats Serbia equals 0.11
The probability that Japan beats Latvia equals 0.10
The probability that Japan beats Netherlands equals 0.30
The probability that Japan beats Poland equals 0.34
The probability that Japan beats ROC equals 0.26
The probability that Japan beats Serbia equals 0.10
The probability that Latvia beats Netherlands e

In [4]:
def simulate_group(print_example):
    wins = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
    for i, game in enumerate(games):
        if random.random() < probabilities[i]:
            if print_example:
                print(game[0] + " beats " + game[1])
            wins[game[0]] += 1
        else: 
            if print_example:
                print(game[1] + " beats " + game[0])
            wins[game[1]] += 1
    # We break ties in wins randomly (in reality ties were broken based on scored points)
    classification = sorted(wins.items(), key=lambda x: (x[1], random.random()), reverse=True)
    if print_example:
        print(classification)
    return classification   

In [5]:
def simulate_bracket(classification, print_example):
    # 3rd seed plays 6th seed
    if random.random() < ranking_2_probability(classification[2][0],classification[5][0]):
        winner_3_6 = classification[2][0]
        if print_example:
            print(classification[2][0] + " beats " + classification[5][0] + " in the first quarter-final")
    else:
        winner_3_6 = classification[5][0]
        if print_example:
            print(classification[5][0] + " beats " + classification[2][0] + " in the first quarter-final")
    # 4th seed plays 5th seed
    if random.random() < ranking_2_probability(classification[3][0],classification[4][0]):
        winner_4_5 = classification[3][0]
        if print_example:
            print(classification[3][0] + " beats " + classification[4][0] + " in the second quarter-final")
    else:
        winner_4_5 = classification[4][0]
        if print_example:
            print(classification[4][0] + " beats " + classification[3][0] + " in the second quarter-final")
    # 1st seed plays winner of 4th vs 5th seed
    if random.random() < ranking_2_probability(classification[0][0],winner_3_6):
        winner_1st_semi = classification[0][0]
        loser_1st_semi = winner_3_6
    else:
        winner_1st_semi = winner_3_6
        loser_1st_semi = classification[0][0]
    if print_example:
        print(winner_1st_semi + " beats " + loser_1st_semi + " in the first semi-final")
    # 2nd seed plays winner of 3rd vs 4th seed
    if random.random() < ranking_2_probability(classification[1][0],winner_4_5):
        winner_2nd_semi = classification[1][0]
        loser_2nd_semi = winner_4_5
    else:
        winner_2nd_semi = winner_4_5
        loser_2nd_semi = classification[1][0]
    if print_example:
        print(winner_2nd_semi + " beats " + loser_2nd_semi + " in the second semi-final")
    # Final
    if random.random() < ranking_2_probability(winner_1st_semi,winner_2nd_semi):
        gold = winner_1st_semi
        silver = winner_2nd_semi
    else:
        gold = winner_2nd_semi
        silver = winner_1st_semi
    if print_example:
        print(gold + " beats " + silver + " to win gold!")
    # Bronze-medal game
    if random.random() < ranking_2_probability(loser_1st_semi,loser_2nd_semi):
        bronze = loser_1st_semi
        fourth = loser_2nd_semi
    else:
        bronze = loser_2nd_semi
        fourth = loser_1st_semi
    if print_example:
        print(bronze + " beats " + fourth + " to win bronze!")
    return (gold, silver, bronze)   

print_example = True
simulate_bracket(simulate_group(print_example),print_example)

China beats Belgium
Japan beats Belgium
Latvia beats Belgium
Belgium beats Netherlands
Poland beats Belgium
ROC beats Belgium
Serbia beats Belgium
China beats Japan
Latvia beats China
Netherlands beats China
Poland beats China
China beats ROC
Serbia beats China
Latvia beats Japan
Netherlands beats Japan
Poland beats Japan
Japan beats ROC
Serbia beats Japan
Latvia beats Netherlands
Latvia beats Poland
ROC beats Latvia
Latvia beats Serbia
Poland beats Netherlands
ROC beats Netherlands
Serbia beats Netherlands
Poland beats ROC
Serbia beats Poland
Serbia beats ROC
[('Latvia', 6), ('Serbia', 6), ('Poland', 5), ('ROC', 3), ('China', 3), ('Japan', 2), ('Netherlands', 2), ('Belgium', 1)]
Poland beats Japan in the first quarter-final
ROC beats China in the second quarter-final
Latvia beats Poland in the first semi-final
ROC beats Serbia in the second semi-final
Latvia beats ROC to win gold!
Poland beats Serbia to win bronze!


('Latvia', 'ROC', 'Poland')

In [6]:
runs = 100000
inv_runs = 1.0 / float(runs)
print_example = False
prob_gold = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
prob_silver = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
prob_bronze = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
prob_win_group = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
    
for i in range(runs):
    classification = simulate_group(print_example)
    prob_win_group[classification[0][0]] += inv_runs
    (gold, silver, bronze) = simulate_bracket(classification,print_example)
    prob_gold[gold] += inv_runs
    prob_silver[silver] += inv_runs
    prob_bronze[bronze] += inv_runs

print("Probability of gold")
for country in prob_gold:
    print(country + ": {:0.4f}".format(prob_gold[country]))
print("\nProbability of silver")
for country in prob_silver:
    print(country + ": {:0.4f}".format(prob_silver[country]))
print("\nProbability of bronze")
for country in prob_bronze:
    print(country + ": {:0.4f}".format(prob_bronze[country]))
print("\nProbability of winning group stage")
for country in prob_win_group:
    print(country + ": {:0.4f}".format(prob_win_group[country]))

Probability of gold
Belgium: 0.0236
China: 0.0032
Japan: 0.0025
Latvia: 0.4229
Netherlands: 0.0339
Poland: 0.0223
ROC: 0.0618
Serbia: 0.4298

Probability of silver
Belgium: 0.0790
China: 0.0181
Japan: 0.0158
Latvia: 0.2785
Netherlands: 0.1031
Poland: 0.0768
ROC: 0.1505
Serbia: 0.2781

Probability of bronze
Belgium: 0.1122
China: 0.0307
Japan: 0.0264
Latvia: 0.1913
Netherlands: 0.1454
Poland: 0.1100
ROC: 0.1945
Serbia: 0.1896

Probability of winning group stage
Belgium: 0.0219
China: 0.0029
Japan: 0.0025
Latvia: 0.4282
Netherlands: 0.0313
Poland: 0.0211
ROC: 0.0571
Serbia: 0.4351


In [12]:
runs = 1000000
selected_runs = 0
print_example = False
counts_gold = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
counts_silver = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
counts_bronze = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
counts_win_group = {"Belgium" : 0, "China" : 0, "Japan" : 0, "Latvia" : 0, "Netherlands" : 0, "Poland": 0, "ROC": 0, "Serbia" : 0}
    
for i in range(runs):
    classification = simulate_group(print_example)
# check if Serbia was 7th or 8th in group
    if classification[6][0] == "Serbia" or classification[7][0] == "Serbia":
        selected_runs += 1
        counts_win_group[classification[0][0]] += 1.0
        (gold, silver, bronze) = simulate_bracket(classification,print_example)
        counts_gold[gold] += 1.0
        counts_silver[silver] += 1.0
        counts_bronze[bronze] += 1.0

print("Selected runs: " + str(selected_runs))
print("Probability of Serbia not making the bracket: {:0.4f}".format(float(selected_runs)/float(runs)))
print("Probability of gold conditioned on Serbia not making the bracket")
for country in prob_gold:
    print(country + ": {:0.4f}".format(counts_gold[country]/selected_runs))
print("\nProbability of silver conditioned on Serbia not making the bracket")
for country in prob_silver:
    print(country + ": {:0.4f}".format(counts_silver[country]/selected_runs))
print("\nProbability of bronze conditioned on Serbia not making the bracket")
for country in prob_bronze:
    print(country + ": {:0.4f}".format(counts_bronze[country]/selected_runs))
print("\nProbability of winning group stage conditioned on Serbia not making the bracket")
for country in prob_win_group:
    print(country + ": {:0.4f}".format(counts_win_group[country]/selected_runs))

Selected runs: 5578
Probability of Serbia not making the bracket: 0.0056
Probability of gold conditioned on Serbia not making the bracket
Belgium: 0.0665
China: 0.0117
Japan: 0.0081
Latvia: 0.6284
Netherlands: 0.0855
Poland: 0.0604
ROC: 0.1395
Serbia: 0.0000

Probability of silver conditioned on Serbia not making the bracket
Belgium: 0.1576
China: 0.0532
Japan: 0.0432
Latvia: 0.1612
Netherlands: 0.1857
Poland: 0.1501
ROC: 0.2490
Serbia: 0.0000

Probability of bronze conditioned on Serbia not making the bracket
Belgium: 0.1665
China: 0.0563
Japan: 0.0586
Latvia: 0.1518
Netherlands: 0.1942
Poland: 0.1658
ROC: 0.2067
Serbia: 0.0000

Probability of winning group stage conditioned on Serbia not making the bracket
Belgium: 0.0592
China: 0.0127
Japan: 0.0084
Latvia: 0.6674
Netherlands: 0.0753
Poland: 0.0570
ROC: 0.1199
Serbia: 0.0000
